---
title: "15-SAS 对二项分布的概率计算与假设检验"
author: "Simon Zhou"
date: "2025-06-07"
date-modified: "2025-06-08"
format: 
    html:
        code-fold: true
        code-line-numbers: true
        code-highlight: true
        fig_caption: true
        number-sections: true
        toc: true
        toc-depth: 3
---

In [1]:
%load_ext saspy.sas_magic

# SAS 与 Binomial

在 SAS 中,与二项分布有关的函数为 `probbnm(p,n,r)`,函数中 `p` 为某事件的发生概率,`n` 为样本含量，为阳性事件的例数,该函数可以计算出发生阳性事件的例数从 `0` 到 `r` 的累计概率。利用该函数可以对服从二项分布的数据进行概率计算和假设检验。

# 阳性事件发生的概率

某种医学技能测试的通过率为0.80。今有10例学生参加测试,试分别计算这10例学生中有6、7、8人获得通过的概率。本例 π=0.8,n=10,计算 r=6、7、8的概率.

In [2]:
%%SAS
/*程序15-1*/
data binomial15_1;
    do r=6 to 8;
        p1=probbnml(0.8,10,r);
        p2=probbnml(0.8,10,r-1);
        p=p1-p2;
        output;
    end;
proc print;
    var r p;
run;

Using SAS Config named: winlocal
SAS Connection established. Subprocess id is 30048



观测,r,p
1,6,0.08808
2,7,0.20133
3,8,0.30199


# 总体率的区间估计(正态近似法)

根据数理统计学的中心极限定理可得,当 n 较大、π 不接近 0 也不接近 1 时,二项分布 $B(n,π)$ 近似正态分布 $N[n \pi,nπ(1-π)]$,相应的样本率 p 的分布也近似正态分布 $N(\pi,\sigma_{p}^2)$。为此,当 n 较大、p 和 1-p 均不太小,如 np 和 n(1-p) 均>5时,可利用样本率 p 的分布近似正态分布来估计总体率的 1-α 置信区间。

In [3]:
%%SAS
data binomial15_2;
    n=100;
    p=0.45;
    sp=sqrt(p*(1-p)/n);
    u=probit(0.975);
    usp=u*sp;
    lclm=p-usp;
    uclm=p+usp;
proc print;
    var n p sp lclm uclm;
run;

观测,n,p,sp,lclm,uclm
1,100,0.45,0.049749,0.35249,0.54751


## 程序说明

数据集中 n 为观察的患者人数 p 为样本率,驴为率的标准误,u 为置信水准为 0.05 时标准正态分布的双侧界值, lclm 为 95% 置信区间的下限,uclm 为 95% 置信区间的上限。

# 样本率与总体率的比较(直接法)

## 示例1

已知输卵管结扎的育龄妇女实施壶腹部-壶腹部吻合术后的受孕率为 0.55。今对 10例输卵管结扎的育龄妇女实施峡部-峡部吻合术,结果有9 人受孕。问实施峡部-峡部吻合术妇女的受孕率是否高于壶腹部-壶腹部吻合术的受孕率?

本例 π=0.55,n=10,r=9。

## 程序说明

本例为单侧检验,首先用函数计算发生例数≤8的累计概率d,再计算1-d就是发生例数>9的概率。

In [2]:
%%SAS
/*程序15-3*/
data binomial15_3;
    d= probbnml(0.55,10,8);
    p= 1-d;
run;
proc print;
    var p;
run;

Using SAS Config named: winlocal
SAS Connection established. Subprocess id is 11632



观测,p
1,0.023257


## 结果说明

由于 P=0.023 257<0.05,说明样本率与总体率的差别有统计学意义,可认为行峡部-峡部吻合术的受孕率要高于壶腹部-壶腹部吻合术。

## 示例2

已知某高校临床医学专业一年级女生100m短跑的达标率为0.70。现在该校一年级的预防医学专业中随机测试了10例女生,有9人达标。问该校这两个专业一年级女生 100m 短跑的达标率是否不同?

本例 π=0.70,n=10,r=9。

### 解题

这是一个**小样本单个比例的假设检验问题**，可以使用**二项分布检验（Binomial test）**。

1. 原假设与备择假设

* $H_0$：预防医学专业的达标率 = 0.70（与临床医学相同）
* $H_1$：预防医学专业的达标率 ≠ 0.70（与临床医学不同）

这是一个**双侧检验（two-sided test）**。

2. 检验统计量

我们用 **二项分布** 进行精确检验：

已知：

* 样本容量 $n = 10$
* 达标人数 $r = 9$
* 总体比例 $\pi = 0.70$

我们计算**在 $n=10$, $\pi=0.70$** 时，得到“像 9 人或比 9 人更极端”的概率（双尾 p 值）：

3. 二项分布计算公式

单个值的概率为：

$$
P(R = r) = \binom{n}{r} \cdot \pi^r \cdot (1 - \pi)^{n - r}
$$

具体来说：

$$
P(R = 9) = \binom{10}{9} \cdot 0.7^9 \cdot 0.3^1
$$

$$
P(R = 10) = \binom{10}{10} \cdot 0.7^{10}
$$

4. 双尾 p 值的计算

我们要计算：

$$
p = P(R \geq 9) + P(R \leq x)，
$$

其中 $P(R \leq x)$ 使得 $P(R \leq x) \leq P(R \geq 9)$。我们可以查二项分布表或用软件计算。

5. 用 R 语言求解示例

```r
# 双侧二项检验
binom.test(x = 9, n = 10, p = 0.70, alternative = "two.sided")
```

6. 用 Pytho 求解示例

```python
binomtest(k=9, n=10, p=0.70, alternative='two-sided')
```

具体运行代码及结果如下所示。

### 总结公式

* $H_0: p = 0.70$
* 使用二项分布：

  $$
  P(r) = \binom{n}{r} \cdot p^r \cdot (1 - p)^{n - r}
  $$
* 双侧 p 值为：

  $$
  p = P(X \geq r) + P(X \leq x), \text{ 使得 } P(X \leq x) \leq P(X \geq r)
  $$



In [4]:
%%SAS
/*程序15-4*/
data binomial15_4;
    p01=probbnml(0.7,10,9);
    p02=probbnml(0.7,10,8);
    p0=p01-p02;
    do i=0 to 10;
        p11=probbnml(0.7,10,i);
        p12=probbnml(0.7,10,i-1);
    pl=p11-p12;
        if i=0 then pl=p11;
    if pl<=p0 then output;
    end;
run;
proc means sum;
    var pl;
run;

分析变量: pl
总和
0.2995767


### 程序说明

首先用函数计算发生例数≤9的累计概率 p01,以及发生例数<8的累计概率 p02，p0 就是发生例数 =9 的概率,由于本例是双侧检验,还需要分别计算发生例数=i(i=0,1，…,10)的概率,考虑比发生例数 =9更背离无效假设(即两个专业达标率相同)的事件,即满足p1≤p0,计算这些事件的概率之和,所得即为无效假设成立的概率。

### 结果说明

由于 P=0.299 576 7>0.05,说明尚不能认为两个样本率的差别有统计学意义,即不能认为两个专业一年级女生 100m 短跑的达标率不同

In [5]:
#程序15-5
from scipy.stats import binomtest

# 参数设置
r = 9        # 达标人数
n = 10       # 总人数
p0 = 0.70    # 临床医学专业达标率

# 执行双侧二项检验
result = binomtest(k=r, n=n, p=p0, alternative='two-sided')

# 输出结果
print("p值 =", result.pvalue)
print("是否显著差异（alpha=0.05）:", "显著" if result.pvalue < 0.05 else "不显著")


p值 = 0.2995766784999999
是否显著差异（alpha=0.05）: 不显著


# 样本率与总体率的比较(正态近似法)

根据二项分布的性质,当n较大、p和1-p均不太小,如和n(1-p)均>5时,可用正态分布来近似。下例介绍用正态近似法完成样本率和总体率的比较。

## 示例

已知某疾病采用常规治疗的治愈率约为45%。现随机抽取180例该疾病患者改用新的治疗方法进行治疗,治愈117人。问新治疗方法是否比常规治疗方法的效果好?

本例 π=0.45,n=180,x=117。

### 求解

使用正态近似法（Z 检验对单个比例进行假设检验的公式如下：

1. 假设检验设定

我们要检验的原假设和备择假设是：

* $H_0: p = p_0$（治愈率与常规治疗相同，即 0.45）
* $H_1: p > p_0$（新治疗方法更好，即治愈率更高）

这是一个**右尾单侧检验**。

2. 正态近似检验的统计量公式

当样本量较大时（np 和 n(1−p) 都 ≥ 5），可以用正态分布近似二项分布。检验统计量为：

$$
Z = \frac{\hat{p} - p_0}{\sqrt{\frac{p_0 (1 - p_0)}{n}}}
$$

其中：

* $\hat{p} = \frac{x}{n}$：样本比例（新疗法下的治愈率）
* $p_0$：原假设下的比例（常规疗法治愈率）
* $n$：样本容量
* $x$：样本中成功次数

3. 代入数据计算

已知：

* $x = 117$
* $n = 180$
* $\hat{p} = \frac{117}{180} = 0.65$
* $p_0 = 0.45$

计算标准误差（SE）：

$$
SE = \sqrt{\frac{0.45 \times (1 - 0.45)}{180}} = \sqrt{\frac{0.2475}{180}} \approx 0.03713
$$

计算Z值：

$$
Z = \frac{0.65 - 0.45}{0.03713} \approx \frac{0.20}{0.03713} \approx 5.39
$$

4. 计算P值，查表或使用函数计算。

In [6]:
%%SAS
/*程序15-6*/
data binomial15_6;
    n=180;
    x=117;
    pai= 0.45;
    p = x/n;
    u = (p-pai)/sqrt(pai*(1-pai)/n);
    pro = (1-probnorm(abs(u)))*2;
run;
proc print;
    var u pro;
run;

观测,u,pro
1,5.39360,6.906E-8


### 程序说明

数据集中 `n` 为样本例数,`x` 为治愈例数,`pai`为总体率,`p` 为样本率,`u` 为检验统计量，`pro` 为u所对应的概率值。这里用到了标准正态函数 `probnorm`。

### 结果说明

“检验的检验统计量的值为 5.393 60,所对应的 P值为 6 906x10-,远远<0.05,说明样本率和总体率之间的差异有统计学意义,可以认为新治疗方法比常规疗法的效果好

# 两个样本率比较的 u检验

两样本率的比较,目的在于对相应的两总体率进行统计推断。可利用样本率的分布近似正态分布,以及两个独立的正态变量之差也服从正态分布的性质,采用正态近似法对两总体率做出统计推断。

In [10]:
%%SAS
/*程序15_7*/
data binomial15_7;
    n1 = 120;
    n2 = 110;
    x1 = 36;
    x2 = 22;
    p1 = x1/n1;
    p2 = x2/n2;
    pc = (x1+x2)/(n1+n2);
    sp = sqrt(pc*(1-pc)*(1/n1+1/n2));
    u = (p1-p2)/sp;
    p = (1-probnorm(abs(u)))*2;
    format u p 8.4;
run;
proc print;
    var pc sp u p;
run;

观测,pc,sp,u,p
1,0.25217,0.057323,1.7445,0.0811


### 程序说明

数据集中 n1 和 n2 分别为男性和女性的调査人数,x1 和 x2 分别为患病人数,pc为总发病率,p1 和 p2 表示男性和女性各自的发病率,为合并标准误,u为统计量,p为u所对应的概率值。

### 结果说明

由于本例检验统计量 u=1.744 5,P=0.081 1>0.05,说明两个样本率的差别无统计学意义,所以尚不能认为该职业人群颈椎病的发病有性别差异.